<a href="https://colab.research.google.com/github/aspectxlol/uprak-pos/blob/main/UPRAK_POS_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem POS UPRAK - Point of Sale Sekolah
Sistem kasir lengkap dengan manajemen produk, keranjang belanja, checkout, dan dukungan pembayaran kode QR.

## 1. Instal Dependensi yang Diperlukan
Sel ini menginstal pustaka Python yang diperlukan agar aplikasi berfungsi dengan benar.
- `qrcode[pil]`: Digunakan untuk menghasilkan kode QR untuk pembayaran.
Jalankan sel ini sekali untuk memastikan semua dependensi terinstal di lingkungan Anda.

In [ ]:
%pip install qrcode[pil]

## 2. Impor Modul yang Diperlukan
Sel ini mengimpor semua pustaka yang diperlukan untuk aplikasi.
- **Pustaka Sistem:** `os`, `sys`, `subprocess` untuk interaksi sistem.
- **Penanganan Data:** `csv` untuk penyimpanan produk, `json` dan `base64` untuk penyandian data.
- **Utilitas:** `datetime` untuk stempel waktu, `pathlib` untuk jalur file.
- **Pustaka Eksternal:** `qrcode` untuk menghasilkan kode pembayaran QR.
- **Tampilan Notebook:** `clear_output` diimpor untuk menangani pembersihan layar dalam Jupyter Notebook dengan benar.

In [ ]:
import os
import csv
import subprocess
import sys
import json
import base64
import time
from datetime import datetime
from pathlib import Path
import qrcode
from IPython.display import clear_output

print("✓ All imports successful")

## 3. Fungsi Utilitas dan Konstanta Warna
Sel ini mendefinisikan fungsi pembantu dan konstanta yang digunakan di seluruh aplikasi untuk meningkatkan antarmuka pengguna dan interaksi.
- **Konstanta Warna:** Variabel `COLOR_...` menyimpan kode escape ANSI untuk mewarnai keluaran teks.
- **`color(text, code)`**: Menerapkan warna pada teks yang diberikan.
- **`clear_screen()`**: Membersihkan sel keluaran untuk menjaga antarmuka tetap bersih (menggunakan `clear_output` untuk notebook).
- **`pause()`**: Menjeda eksekusi hingga pengguna menekan Enter.
- **`input_number()` & `input_int()`**: Fungsi pembantu untuk mendapatkan input numerik yang divalidasi dari pengguna.
- **`get_timestamp()`**: Mengembalikan tanggal dan waktu saat ini sebagai string.
- **`format_idr()`**: Memformat angka menjadi string mata uang Rupiah Indonesia (misalnya, Rp 10.000).

In [ ]:
# =========================================================
# Color Constants
# =========================================================
COLOR_HEADER = '96'   # Bright cyan
COLOR_MENU = '93'     # Bright yellow
COLOR_OK = '92'       # Bright green
COLOR_ERROR = '91'    # Bright red
COLOR_INPUT = '94'    # Bright blue
COLOR_RESET = '0'

def color(text: str, code: str) -> str:
    """Pembungkus untuk kode warna ANSI di terminal."""
    return f"\033[{code}m{text}\033[0m"

def clear_screen() -> None:
    """Membersihkan layar terminal untuk tampilan yang lebih rapi."""
    # os.system('cls' if os.name == 'nt' else 'clear') # Tidak berfungsi di notebook
    clear_output(wait=True)

def pause(msg: str = "Tekan Enter untuk melanjutkan...") -> None:
    """Menghentikan eksekusi sampai pengguna menekan Enter."""
    sys.stdout.flush()
    time.sleep(0.5)
    input(msg)

def input_text(prompt: str) -> str:
    """Meminta input teks dengan delay untuk stabilitas notebook."""
    sys.stdout.flush()
    time.sleep(0.5)
    return input(prompt)

def input_number(prompt: str, allow_zero: bool = False) -> float:
    """Meminta pengguna memasukkan angka dan terus meminta sampai input valid."""
    while True:
        sys.stdout.flush()
        time.sleep(0.5)
        val = input(prompt)
        try:
            num = float(val)
            if not allow_zero and num <= 0:
                print("Masukkan angka yang positif.")
                continue
            return num
        except ValueError:
            print("Masukan tidak valid. Masukkan angka.")

def input_int(prompt: str, allow_zero: bool = False) -> int:
    """Meminta pengguna memasukkan bilangan bulat dan terus meminta sampai input valid."""
    while True:
        sys.stdout.flush()
        time.sleep(0.5)
        val = input(prompt)
        try:
            num = int(val)
            if not allow_zero and num <= 0:
                print("Masukkan bilangan bulat yang positif.")
                continue
            return num
        except ValueError:
            print("Masukan tidak valid. Masukkan bilangan bulat.")

def get_timestamp() -> str:
    """Mengembalikan tanggal dan waktu saat ini."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def format_idr(amount: float) -> str:
    """Memformat angka menjadi Rupiah Indonesia (IDR)."""
    return f"Rp {int(amount):,}".replace(",", ".")

print("✓ Utility functions loaded")

## 4. Kelas POS - Manajemen Produk dan Keranjang Belanja
Bagian ini mendefinisikan kelas inti `POS` yang mengelola status aplikasi.
- **`__init__`**: Menginisialisasi daftar produk, keranjang belanja, dan memuat produk yang ada dari `products.csv`.
- **`load_products()`**: Membaca produk dari file CSV.
- **`_validate_price()`**: Pembantu internal untuk memastikan input harga valid.
- **`save_products()`**: Menulis daftar produk saat ini kembali ke file CSV.
- **`list_products()`**: Menampilkan tabel terformat dari semua produk yang tersedia.

In [ ]:
class POS:
    """Kelas POS utama untuk mengelola produk, keranjang belanja, dan transaksi."""
    def __init__(self) -> None:
        self.products = []  # List produk: {id, name, price}
        self.cart = []      # List keranjang: {id, name, price, qty}
        self.next_product_id = 1
        self.products_file = "products.csv"
        self.load_products()

    def load_products(self) -> None:
        """Memuat produk dari file CSV jika ada."""
        try:
            with open(self.products_file, newline='', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                self.products = []
                max_id = 0
                for row in reader:
                    prod = {
                        'id': int(row['id']),
                        'name': row['name'],
                        'price': float(row['price'])
                    }
                    self.products.append(prod)
                    if prod['id'] > max_id:
                        max_id = prod['id']
                self.next_product_id = max_id + 1
        except FileNotFoundError:
            self.products = []
            self.next_product_id = 1

    def _validate_price(self, price_str: str) -> bool:
        """Validasi input harga."""
        try:
            price = float(price_str)
            return price > 0
        except ValueError:
            return False

    def save_products(self) -> None:
        """Menyimpan semua produk ke file CSV."""
        with open(self.products_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['id', 'name', 'price'])
            writer.writeheader()
            for p in self.products:
                writer.writerow({'id': p['id'], 'name': p['name'], 'price': p['price']})

    def list_products(self, show_header=True) -> None:
        """Menampilkan semua produk yang tersedia."""
        if show_header:
            print(color("ID  Nama                 Harga (IDR)", COLOR_MENU))
            print(color("--  -------------------- -----------", COLOR_MENU))
        for p in self.products:
            print(f"{p['id']:2}  {p['name'][:20]:20} {format_idr(p['price']):>13}")

print("✓ POS class initialized")

## 5. Metode POS - Tambah Produk
Sel ini menambahkan fungsionalitas `add_product` ke kelas `POS`.
- **`add_product()`**: Memungkinkan pengguna memasukkan nama dan harga untuk produk baru.
    - Memvalidasi bahwa nama tidak kosong dan harga adalah angka positif.
    - Menambahkan produk ke daftar dengan ID unik dan menyimpannya ke file CSV.
    - Menampilkan pesan sukses atau kesalahan menggunakan utilitas pemformatan.

In [ ]:
def add_product(self) -> None:
    """Menambahkan produk baru ke daftar dan menyimpan ke file."""
    clear_screen()
    print(color("=== Tambah Produk ===", COLOR_HEADER))

    try:
        name = input_text("Nama produk: ").strip()
        if not name:
            print(color("Nama tidak boleh kosong.", COLOR_ERROR))
            pause()
            return

        price_str = input_text("Harga produk: ").strip()
        if not self._validate_price(price_str):
            print(color("Harga tidak valid. Masukkan angka positif.", COLOR_ERROR))
            pause()
            return

        price = float(price_str)

        product = {
            'id': self.next_product_id,
            'name': name,
            'price': price
        }
        self.products.append(product)
        self.next_product_id += 1
        self.save_products()
        print(color(f"Produk '{name}' berhasil ditambahkan dengan ID {product['id']}", COLOR_OK))
        pause()
    except KeyboardInterrupt:
        print(color("Dibatalkan. Kembali ke menu utama.", COLOR_ERROR))
        pause()

# Bind method to POS class
POS.add_product = add_product
print("✓ add_product method added")

## 6. Metode POS - Edit dan Manajemen Keranjang
Bagian ini menangani logika belanja inti: mengedit produk, menambahkan item ke keranjang, dan mengelola keranjang.
- **`edit_product()`**: Memungkinkan pengeditan nama dan harga produk yang ada berdasarkan ID-nya.
- **`add_to_cart()`**: Menambahkan produk yang dipilih (berdasarkan ID) ke keranjang belanja dengan jumlah tertentu.
    - Menangani akumulasi jumlah jika produk sudah ada di keranjang.
- **`remove_from_cart()`**: Menghapus item yang dipilih dari keranjang belanja.
- **`show_cart()`**: Menampilkan isi keranjang belanja saat ini dengan sub-total dan total keseluruhan.

In [ ]:
def edit_product(self) -> None:
    """Mengedit produk yang ada dengan ID dan menyimpan ke file."""
    clear_screen()
    print(color("=== Edit Produk ===", COLOR_HEADER))
    if not self.products:
        print(color("Tidak ada produk untuk diedit.", COLOR_ERROR))
        pause()
        return

    try:
        self.list_products()
        pid_input = input_text("Masukkan ID produk untuk diedit: ").strip()
        if not pid_input.isdigit():
            print(color("ID harus berupa angka.", COLOR_ERROR))
            pause()
            return

        pid = int(pid_input)
        product = next((p for p in self.products if p['id'] == pid), None)

        if not product:
            print(color("Produk tidak ditemukan.", COLOR_ERROR))
            pause()
            return

        print(f"Mengedit '{product['name']}'")
        new_name = input_text(f"Nama baru (kosongkan untuk tetap '{product['name']}'): ").strip()
        if new_name:
            product['name'] = new_name

        new_price = input_text(f"Harga baru (kosongkan untuk tetap {format_idr(product['price'])}): ").strip()
        if new_price:
            if self._validate_price(new_price):
                product['price'] = float(new_price)
            else:
                print(color("Harga tidak valid.", COLOR_ERROR))

        self.save_products()
        print(color("Produk berhasil diperbarui.", COLOR_OK))
        pause()
    except KeyboardInterrupt:
        print(color("Dibatalkan. Kembali ke menu utama.", COLOR_ERROR))
        pause()

def add_to_cart(self) -> None:
    """Menambahkan produk ke keranjang berdasarkan ID dan jumlah."""
    clear_screen()
    print(color("=== Tambah ke Keranjang ===", COLOR_HEADER))
    if not self.products:
        print(color("Tidak ada produk yang tersedia.", COLOR_ERROR))
        pause()
        return

    try:
        self.list_products()
        pid_input = input_text("Masukkan ID produk untuk ditambahkan: ").strip()
        if not pid_input.isdigit():
            print(color("ID harus berupa angka.", COLOR_ERROR))
            pause()
            return

        pid = int(pid_input)
        product = next((p for p in self.products if p['id'] == pid), None)

        if not product:
            print(color("Produk tidak ditemukan.", COLOR_ERROR))
            pause()
            return

        qty_input = input_text("Jumlah: ").strip()
        if not qty_input.isdigit() or int(qty_input) <= 0:
            print(color("Jumlah tidak valid.", COLOR_ERROR))
            pause()
            return

        qty = int(qty_input)
        cart_item = next((c for c in self.cart if c['id'] == pid), None)
        if cart_item:
            cart_item['qty'] += qty
        else:
            self.cart.append({
                'id': product['id'],
                'name': product['name'],
                'price': product['price'],
                'qty': qty
            })
        print(color(f"Berhasil menambahkan {qty} x {product['name']} ke keranjang.", COLOR_OK))
        pause()
    except KeyboardInterrupt:
        print(color("Dibatalkan. Kembali ke menu utama.", COLOR_ERROR))
        pause()

def remove_from_cart(self) -> None:
    """Menghapus item dari keranjang berdasarkan ID produk."""
    clear_screen()
    print(color("=== Hapus dari Keranjang ===", COLOR_HEADER))
    if not self.cart:
        print(color("Keranjang kosong.", COLOR_ERROR))
        pause()
        return

    try:
        self.show_cart()
        pid_input = input_text("Masukkan ID produk untuk dihapus: ").strip()
        if not pid_input.isdigit():
            print(color("ID harus berupa angka.", COLOR_ERROR))
            pause()
            return

        pid = int(pid_input)
        idx = next((i for i, c in enumerate(self.cart) if c['id'] == pid), None)

        if idx is None:
            print(color("Item tidak ditemukan di keranjang.", COLOR_ERROR))
            pause()
            return

        removed = self.cart.pop(idx)
        print(color(f"Berhasil menghapus {removed['name']} dari keranjang.", COLOR_OK))
        pause()
    except KeyboardInterrupt:
        print(color("Dibatalkan. Kembali ke menu utama.", COLOR_ERROR))
        pause()

def show_cart(self) -> None:
    """Menampilkan isi keranjang belanja yang sedang aktif."""
    if not self.cart:
        print(color("Keranjang kosong.", COLOR_ERROR))
        return
    print(color("ID  Nama                 Qty  Harga (IDR)   Subtotal (IDR)", COLOR_MENU))
    print(color("--  -------------------- --- -----------   ---------------", COLOR_MENU))
    total = 0
    for c in self.cart:
        subtotal = c['qty'] * c['price']
        total += subtotal
        print(f"{c['id']:2}  {c['name'][:20]:20} {c['qty']:3} {format_idr(c['price']):>13}  {format_idr(subtotal):>14}")
    print(f"\nTotal: {format_idr(total)}")

# Bind methods to POS class
POS.edit_product = edit_product
POS.add_to_cart = add_to_cart
POS.remove_from_cart = remove_from_cart
POS.show_cart = show_cart
print("✓ Cart operation methods added")

## 7. Metode POS - Checkout dan Struk (Receipt)
Bagian ini mengimplementasikan langkah-langkah transaksi akhir.
- **`generate_receipt()`**: Membuat struk file teks terformat untuk transaksi.
    - Menyertakan tanggal, nama pelanggan, metode pembayaran, item, dan total.
    - Menyimpan struk ke file bernama `struk_YYYYMMDD_HHMMSS.txt`.
- **`display_qr_code()`**: Menghasilkan dan menampilkan Kode QR untuk pembayaran QRIS.
    - Menggunakan pustaka `qrcode` untuk mengkodekan data pembayaran menjadi gambar yang dapat dipindai.
- **`checkout()`**: Mengelola alur checkout lengkap.
    - Memungkinkan pengguna memilih pembayaran **Tunai** atau **QRIS**.
    - Untuk Tunai: Menghitung kembalian.
    - Untuk QRIS: Menampilkan kode QR dan menunggu pembayaran.
    - Membersihkan keranjang setelah checkout berhasil.

In [ ]:
def generate_receipt(self, total, cash, change, customer_name: str = "Guest", payment_method: str = "Cash") -> None:
    """Membuat dan menyimpan struk transaksi ke file teks."""
    timestamp = get_timestamp()
    filename = f"struk_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("STRUK PEMBELIAN - SCHOOL POS\n")
        f.write(f"Tanggal: {timestamp}\n")
        f.write(f"Pelanggan: {customer_name}\n")
        f.write(f"Metode Pembayaran: {payment_method}\n\n")
        f.write("Barang Belanja:\n")
        f.write("ID  Nama                 Qty  Harga (IDR)   Subtotal (IDR)\n")
        f.write("--  -------------------- --- -----------   ---------------\n")
        for c in self.cart:
            subtotal = c['qty'] * c['price']
            f.write(f"{c['id']:2}  {c['name'][:20]:20} {c['qty']:3} {format_idr(c['price']):>13}  {format_idr(subtotal):>14}\n")
        f.write(f"\nTotal Harga:   {format_idr(total)}\n")
        if payment_method == "Tunai":
            f.write(f"Uang Masuk:    {format_idr(cash)}\n")
            f.write(f"Kembalian:     {format_idr(change)}\n")
        else:
            f.write(f"Dibayar melalui {payment_method}\n")
    print(color(f"Struk berhasil disimpan sebagai {filename}", COLOR_OK))

def display_qr_code(self, amount: float, customer_name: str = "Guest") -> None:
    """Menampilkan kode QR di terminal untuk pembayaran QRIS."""
    try:
        payment_data = {
            "merchantName": "Ujian Praktek",
            "customerName": customer_name,
            "price": str(int(amount))
        }
        encoded_data = base64.b64encode(json.dumps(payment_data).encode()).decode()
        payment_url = f"https://aspectxlol.vercel.app/uprak-pos/payment?data={encoded_data}"

        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=1,
            border=1,
        )
        qr.add_data(payment_url)
        qr.make(fit=True)
        ascii_qr = qr.get_matrix()

        print(color("\n╔════════════════════════════════════╗", COLOR_OK))
        print(color("║    PEMBAYARAN QRIS - PINDAI PONSEL  ║", COLOR_OK))
        print(color("╚════════════════════════════════════╝", COLOR_OK))
        print()

        for row in ascii_qr:
            line = ""
            for cell in row:
                line += "██" if cell else "  "
            print(color(line, COLOR_OK))

        print()
        print(color("╔════════════════════════════════════╗", COLOR_OK))
        print(color(f"║ Nominal: {format_idr(amount)[:24].ljust(24)} ║", COLOR_OK))
        print(color("╚════════════════════════════════════╝", COLOR_OK))
    except Exception as e:
        print(color(f"Kesalahan saat membuat kode QR: {e}", COLOR_ERROR))

def checkout(self) -> None:
    """Menangani proses checkout: pembayaran, kembalian, dan pembuatan struk."""
    clear_screen()
    print(color("=== Checkout ===", COLOR_HEADER))
    if not self.cart:
        print(color("Keranjang kosong.", COLOR_ERROR))
        pause()
        return

    try:
        customer_name = input_text("Nama pelanggan (atau tekan Enter untuk 'Guest'): ").strip() or "Guest"

        self.show_cart()
        total = sum(c['qty'] * c['price'] for c in self.cart)

        print(color("\n--- Metode Pembayaran ---", COLOR_MENU))
        print("1. Tunai")
        print("2. QRIS")

        while True:
            method_input = input_text("Pilih metode (1 atau 2): ").strip()
            if method_input in ['1', '2']:
                break
            print(color("Pilihan tidak valid.", COLOR_ERROR))

        payment_method = "Tunai" if method_input == '1' else "QRIS"

        if payment_method == "Tunai":
            while True:
                cash_input = input_text(f"Uang yang diterima (total {format_idr(total)}): ").strip()
                try:
                    cash = float(cash_input)
                    if cash >= total:
                        break
                    print(color("Uang tidak cukup.", COLOR_ERROR))
                except ValueError:
                    print(color("Input tidak valid.", COLOR_ERROR))

            change = cash - total
            print(color(f"Kembalian: {format_idr(change)}", COLOR_OK))
            self.generate_receipt(total, cash, change, customer_name, payment_method)
        else:
            print(color("\nPembayaran QRIS", COLOR_MENU))
            self.display_qr_code(total, customer_name)
            input_text("Tekan Enter setelah pembayaran berhasil...")
            self.generate_receipt(total, total, 0, customer_name, payment_method)

        print(color("Struk berhasil dibuat.", COLOR_OK))
        self.cart.clear()
        pause()
    except KeyboardInterrupt:
        print(color("Dibatalkan. Kembali ke menu utama.", COLOR_ERROR))
        pause()

POS.generate_receipt = generate_receipt
POS.display_qr_code = display_qr_code
POS.checkout = checkout
print("✓ Checkout and receipt methods added")

## 8. Menu Utama dan Loop Aplikasi
Fungsi-fungsi ini mengontrol alur aplikasi utama dan antarmuka pengguna.
- **`main_menu()`**: Menampilkan operasi yang tersedia kepada pengguna dan mendapatkan pilihan mereka.
- **`run_application()`**: Loop utama yang:
    - Menampilkan menu.
    - Menangkap pilihan pengguna.
    - Menjalankan metode yang sesuai dari kelas `POS` (misalnya, `add_product()`, `checkout()`).
    - Menangani penutupan program dan input yang tidak valid.
    - Terus berulang hingga pengguna memilih untuk keluar.

In [ ]:
def main_menu() -> str:
    """Menampilkan menu utama dengan TUI berdasarkan text input."""
    clear_screen()

    width = 80
    print(color("╔" + "═" * (width - 2) + "╗", COLOR_HEADER))
    header = "═══ SISTEM POS SEKOLAH ═══"
    header_x = (width - len(header) - 2) // 2
    print(color("║" + " " * header_x + header + " " * (width - header_x - len(header) - 2) + "║", COLOR_HEADER))
    print(color("╠" + "═" * (width - 2) + "╣", COLOR_HEADER))

    menu_items = [
        "1. Tambah Produk",
        "2. Edit Produk",
        "3. Lihat Daftar Produk",
        "4. Tambah ke Keranjang",
        "5. Hapus dari Keranjang",
        "6. Lihat Keranjang",
        "7. Checkout",
        "0. Keluar"
    ]

    for item in menu_items:
        print(color("║ " + item.ljust(width - 4) + " ║", COLOR_HEADER))

    print(color("╠" + "═" * (width - 2) + "╣", COLOR_HEADER))
    print(color("║ Masukkan angka pilihan (0-7)" + " " * (width - 32) + "║", COLOR_INPUT))
    print(color("╚" + "═" * (width - 2) + "╝", COLOR_HEADER))

    try:
        choice = input_text("\nPilihan: ").strip()
        return choice if choice in ['0', '1', '2', '3', '4', '5', '6', '7'] else 'x'
    except KeyboardInterrupt:
        return '0'

def run_application() -> None:
    """Perulangan utama aplikasi POS."""
    pos = POS()
    while True:
        choice = main_menu()
        if choice == '1':
            pos.add_product()
        elif choice == '2':
            pos.edit_product()
        elif choice == '3':
            clear_screen()
            print(color("=== Daftar Produk ===", COLOR_HEADER))
            if not pos.products:
                print(color("Tidak ada produk yang tersedia.", COLOR_ERROR))
            else:
                pos.list_products()
            pause()
        elif choice == '4':
            pos.add_to_cart()
        elif choice == '5':
            pos.remove_from_cart()
        elif choice == '6':
            clear_screen()
            print(color("=== Keranjang ===", COLOR_HEADER))
            pos.show_cart()
            pause()
        elif choice == '7':
            pos.checkout()
        elif choice == '0':
            clear_screen()
            print(color("Keluar dari POS. Sampai jumpa!", COLOR_HEADER))
            break
        else:
            print(color("Pilihan tidak valid. Masukkan angka 0-7.", COLOR_ERROR))
            pause()

print("✓ Functions ready")

## 9. Jalankan Aplikasi
Jalankan sel di bawah ini untuk memulai aplikasi POS!
- Periksa keluaran terminal di mana Anda akan melihat menu.
- Gunakan **kotak input** di bagian atas layar untuk berinteraksi.
- Ketik `0` untuk **Keluar** setelah selesai.
- Ketik angka `1` hingga `7` untuk opsi lainnya.
    - Contoh: `1` untuk **Tambah Produk**, `4` untuk **Tambah ke Keranjang**, lalu `7` untuk **Checkout**.

**Catatan:** Jika menggunakan Google Colab, Anda harus menghapus tanda komentar pada baris `run_application()`.

In [ ]:
# Uncomment the line below to run the application
run_application()

print("POS System is ready to use!")
print("To start the application in Google Colab, uncomment and run: run_application()")